In [ ]:
import logging
import yaml
import matplotlib.pyplot as plt
from scipy.stats.distributions import lognorm
import numpy as np
import pandas as pd
import os.path
import pyrheautils as pu
from collections import deque, defaultdict
from facilitybase import CareTier
from tools_util import readModelInputs, getFacDict
import phacsl.utils.formats.csv_tools as csv_tools
import math


In [ ]:
runDesc = '/home/welling/git/pyRHEA_github/src/sim/week_run_OC.yaml'

inputDict = readModelInputs(runDesc)
facDict = getFacDict(inputDict)

totD = defaultdict(int)
nFacD = defaultdict(int)
for fac, rec in facDict.items():
    nFacD[rec['category']] += 1
    totD[rec['category']] += rec['meanPop']['value']
print 'totD: ', totD
print 'nFacD: ', nFacD

# Drop annoying community entries
dropL = []
for abbrev, rec in facDict.items():
    if rec['category'] == 'COMMUNITY':
        dropL.append(abbrev)
for abbrev in dropL:
    del facDict[abbrev]
print facDict.keys()

In [ ]:
mMinusFName = pu.pathTranslate('$(MODELDIR)/Updated_MRSA-Stratified_LOS_Distributions_for_RHEA_2.0_-_10-15-2018_MSRAMINUS_LOS_LE365.csv')
with open(mMinusFName, 'rU') as f:
          mMinusKeys, mMinusRecs = csv_tools.parseCSV(f)
print mMinusKeys
mMinusD = {rec['CODE']: rec for rec in mMinusRecs}

In [ ]:
mPlusFName = pu.pathTranslate('$(MODELDIR)/Updated_MRSA-Stratified_LOS_Distributions_for_RHEA_2.0_-_10-15-2018_MSRAPLUS_LOS_LE365.csv')
with open(mPlusFName, 'rU') as f:
          mPlusKeys, mPlusRecs = csv_tools.parseCSV(f)
print mPlusKeys
mPlusD = {rec['CODE']: rec for rec in mPlusRecs}

In [ ]:
abbrev = 'FVRH'
losMu = mMinusD[abbrev]['LN_MEAN']
losSigma = mMinusD[abbrev]['LN_STD DEV']
losRV = lognorm(losSigma, scale=math.exp(losMu), loc=0.0)
print losRV.mean()

In [ ]:
hospNames = [abbrev for abbrev in mMinusD
             if mMinusD[abbrev]['LN_MEAN'] != '' and facDict[abbrev]['category'] == 'HOSPITAL']
hospNames.sort()
print hospNames
ltacNames = [abbrev for abbrev in mMinusD
             if mMinusD[abbrev]['LN_MEAN'] != '' and facDict[abbrev]['category'] == 'LTAC']
ltacNames.sort()
print ltacNames

In [ ]:
for fac in hospNames + ltacNames:
    losMu = mMinusD[fac]['LN_MEAN']
    losSigma = mMinusD[fac]['LN_STD DEV']
    mMinusD[fac]['crv'] = lognorm(losSigma, scale=math.exp(losMu), loc=0.0)

    losMu = mPlusD[fac]['LN_MEAN']
    losSigma = mPlusD[fac]['LN_STD DEV']
    mPlusD[fac]['crv'] = lognorm(losSigma, scale=math.exp(losMu), loc=0.0)


In [ ]:
NSAMP = 5

def genWts(dct, nameL):
    nNm = len(nameL)
    nTot = sum([dct[fac]['N'] for fac in nameL])
    wts = np.zeros(nNm)
    for idx, fac in enumerate(nameL):
        wts[idx] = float(dct[fac]['N'])/float(nTot)
    return wts

nPlusWts = genWts(mPlusD, hospNames)
#print nPlusWts
#print sum(nPlusWts)
nMinusWts = genWts(mMinusD, hospNames)
#print sum(nMinusWts)

In [ ]:
randIndices = np.random.choice(len(hospNames), size=NSAMP, p=nPlusWts)
print nPlusSamps
sampA = np.empty((NSAMP, len(hospNames)))
for idx, fac in enumerate(hospNames):
    sampA[:, idx] = mPlusD[fac]['crv'].rvs(NSAMP)
mixV = np.fromiter([sampA[col, randIndices[col]] for col in xrange(NSAMP)], dtype=np.float)
print mixV
#nPlusRVs = [mPlusD[idx]['crv'] for idx in xrange(len(hospNames))]
#samps = 

In [ ]:
def sampleMixture(dct, nameL):
    wts = genWts(dct, nameL)
    randIndices = np.random.choice(len(nameL), size=NSAMP, p=wts)
    sampA = np.empty((NSAMP, len(nameL)))
    for idx, fac in enumerate(nameL):
        sampA[:, idx] = dct[fac]['crv'].rvs(NSAMP)
    mixV = np.fromiter([sampA[col, randIndices[col]] for col in xrange(NSAMP)], dtype=np.float)
    return mixV
#nPlusRVs = [mPlusD[idx]['crv'] for idx in xrange(len(hospNames))]
#samps = 
samps = sampleMixture(mPlusD, hospNames)
#print samps

In [ ]:
NSAMP = 1000000
plt.hist(sampleMixture(mPlusD, hospNames), bins=100, label='MRSA+')
plt.hist(sampleMixture(mMinusD, hospNames), bins=100, label='MRSA-')
plt.title('Hospitals')
plt.legend()
plt.show()

In [ ]:
NSAMP = 1000000
plt.hist(sampleMixture(mPlusD, ltacNames), bins=100, label='MRSA+')
plt.hist(sampleMixture(mMinusD, ltacNames), bins=100, label='MRSA-')
plt.title('LTACs')
plt.legend()
plt.show()

In [ ]:
NSAMP = 1000000
delayV = sampleMixture(mPlusD, hospNames) - sampleMixture(mMinusD, hospNames)
plt.hist(delayV, bins=100)
plt.title('Hospital delay')
plt.show()
print 'mean: ', np.mean(delayV)
print 'stdv: ', np.std(delayV)

In [ ]:
NSAMP = 1000000
delayV = sampleMixture(mPlusD, ltacNames) - sampleMixture(mMinusD, ltacNames)
plt.hist(delayV, bins=100)
plt.title('LTAC delay')
plt.show()
print 'mean: ', np.mean(delayV)
print 'stdv: ', np.std(delayV)